# Example Problem: Estimating a model with MLE

This notebook is an example of how to estimate a model using Maximum Likelihood Estimation (MLE). The model we will estimate is a simple linear regression model. The data generating process is as follows:

The Stata data file `PSID_data.dta` provides you with the PSID used by [Heathcoate, Perri, and Violante](http://www.sciencedirect.com/science/article/pii/S1094202509000659) (*Review of Economic Dynamics*, 2010)}.  The raw data are available [here](https://ideas.repec.org/c/red/ccodes/09-214.html).  I've modified this only to deflate the labor incomes of heads and spouses (`hlabinc` and `wlabinc`) to 2005$.

You need to do the following:
1. Select only male heads of household who are between 25 and 60 years of age and earn wages $>$ \$7/hr.
2. Create indicator and continuous variables as necessary (see model below).
3. Estimate the following model via a Maximum Likelihood Estimator separately for $t$ = 1971, 1980, 1990, 2000:
    $$
      ln(w_{i,t}) = \alpha + \beta_{1}Educ_{i,t} + \beta_{2}Age_{i,t} + \beta_{3}Age^2_{i,t} + \beta_{4}Black_{i,t} + \beta_{6}OtherRace_{i,t} + \varepsilon_{i,t},
    $$
    where:

     * $w_{i,t}$ = wage of individual $i$ in survey year $t$
     * $Educ_{i,t}$ = education in years
     * $Age_{i,t}$ = age in years
     * $Black_{i,t}$, $OtherRace_{i,t}$ = dummy variables for race = Black, Not $\in$ \{White, Black}.
4. Interpret the coefficient $\beta_{1}$.  How do the returns to education change over time in these data?
\end{enumerate}

The following variable definitions are provided for your benefit (the [PSID provides further documentation](https://psidonline.isr.umich.edu)):

* `hlabinc` = annual labor income of the head
* `hannhrs` = annual hours of the head
* `hsex` = gender of the head (1=Male, 2=Female)
* `hrace` = race of the head (1=White, 2=Black, 3=Native American, 4=Asian/Pacific Islander, 5=Hispanic, 6,7=Other)
* `age` = age of the head
* `hyrsed` = years of education of the head


**HINT**: This model is linear.  So you can check your MLE against an OLS estimator to confirm your results.  See this [QuantEcon notebook](https://lectures.quantecon.org/py/ols.html) for a short tutorial on linear regressions in Python.  Also, you may want a bounded optimizer e.g., try the `L-BFGS-B` and `SLSQP` methods in `scipy.optimize.minimize()`.


In [37]:
# Imports
import pandas as pd
import numpy as np
import scipy.optimize as opt
import scipy.stats as stats
import statsmodels.formula.api as smf

## Step 0:  Read in and clean the data

## Maximum Likelihood Estimation

Log wages are modeled as:
$$
ln(w_{i,t}) = \alpha + \beta_{1}Educ_{i,t} + \beta_{2}Age_{i,t} + \beta_{3}Age^2_{i,t} + \beta_{4}Black_{i,t} + \beta_{5}Hispanic_{i,t} + \beta_{6}OtherRace_{i,t} + \varepsilon_{i,t},
$$

With the assumption that $\varepsilon_{i,t} \sim N(0, \sigma^2)$.

In MLE, we want to find the parameters, $\hat{\beta}$ and $\hat{\sigma}$, that maximize the likelihood of observing the outcome (log wages) conditional on the data.  i.e., for each year (which you will do separately for this question) we want to solve:

$$
max_{\hat{\beta}, \hat{\sigma}}\prod_{i=1}^{N}p(y_{i,t}|x_{i};\hat{\beta}, \hat{\sigma})
$$

With the assumption that $\varepsilon$ is distributed normally with mean zero and variance $\sigma^2$, we know the functional form of the conditional probability function:

$$
\prod_{i=1}^{N}\frac{1}{\sqrt{2\pi \sigma^2}} e^{-\frac{\left(y_{i} - \hat{\beta} x_{i}\right)^2}{2 \hat{\sigma}^2}}
$$

We can estimate this, but it's often simpler to deal with the log-likelihood function:

$$
LL = log \left( \prod_{i=1}^{N}\frac{1}{\sqrt{2\pi \hat{\sigma}^2}} e^{-\frac{\left(y_{i} - \hat{\beta} x_{i}\right)^2}{2 \hat{\sigma}^2}} \right)
$$

$$
LL =  \sum_{i=1}^{N}log \left(\frac{1}{\sqrt{2\pi \hat{\sigma}^2}} e^{-\frac{\left(y_{i} - \hat{\beta} x_{i}\right)^2}{2 \hat{\sigma}^2}}\right)
$$

$$
LL = -\frac{N}{2}log(2\pi) - N log(\hat{\sigma}) - \frac{1}{2\hat{\sigma}^2}\sum_{i=1}^{N}\left( y_{i} - \hat{\beta} x_{i}\right)^2
$$

It's this LL function that we will use to estimate $\beta$, $\sigma$.

### Step 1: Define the statistical objective function

The first thing we'll do then is to write out this statistical objective function that is the LL function.

### Step 2: Optimize the statistical objective function

### Step 3: Estimate with OLS and compare